In [1]:
import numpy as np
#in case we need to repeat experiment
#np.random.seed(255)

import pandas as pd
pd.options.display.max_rows = 22

import matplotlib.pyplot as plt
plt.style.use('classic')

import seaborn as sns
sns.set()

#sklearn imports
from sklearn.cluster import KMeans
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from pandas.api.types import CategoricalDtype
from sklearn.naive_bayes import GaussianNB
import pickle # allows for model to be saved/load to file
import time

#Use print instead of display when run as python script
pyscript = True

#Classifier verborsity where supported
verbose_level=3

#Multiclass classification, binary if falase
multiclass = False
over_sample = True

#inputfile = 'CKME136X10_2018_Data_CTF.csv'
if multiclass:
    inputfile_train_O = 'CKME136X10_2018_Data_CTFB_M_O_Train.csv'
    inputfile_train_U = 'CKME136X10_2018_Data_CTFB_M_U_Train.csv'
    inputfile_test = 'CKME136X10_2018_Data_CTFB_M_Test.csv'
else:
    inputfile_train_O = 'CKME136X10_2018_Data_CTFB_B_O_Train.csv'
    inputfile_train_U = 'CKME136X10_2018_Data_CTFB_B_U_Train.csv'
    inputfile_test = 'CKME136X10_2018_Data_CTFB_B_Test.csv'

if over_sample:
    datafile_train = inputfile_train_O
else:
    datafile_train = inputfile_train_U

datafile_test = inputfile_test
    
model_max_iter = 1000
datestr = 'dec_07_binary_run_1000_KBO'

#Model Store
file_lr = 'lr_' + datestr + '.model'
file_lr_l1 = 'lr_l2_' + datestr + '.model'
file_dt = 'dt_' + datestr + '.model'
file_svm = 'svm_' + datestr + '.model'
file_knn = 'knn_' + datestr + '.model'
file_mlp = 'mlp_' + datestr + '.model'
file_kmean = 'kmean_' + datestr + '.model'
file_nbayes = 'nbayes_' + datestr + '.model'

file_final_train = 'final_train_' + datestr + '.csv'
file_final_test = 'final_test_' + datestr + '.csv'

#Enable Optimization Algorithms
enable_grid_search = False
svm_c = 1
svm_gamma = 1
nfold = 10

enable_lr_l1 = False
predict_lr_l1 = False

# Enable Algorithms
enable_lr = True
enable_dt = True
enable_svm = True
enable_knn = True
enable_mlp = True
enable_nbayes = True

predict_lr = False
predict_dt = True
predict_svm = True
predict_knn = False
predict_mlp = True
predict_nbayes = True

C:\Users\pradeep\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
# this function converts the data frame to the appropriate data type
def convert_type(data):
    data = data.astype('category')
    data['C_MNTH'] = data['C_MNTH'].astype(CategoricalDtype(ordered=True))
    data['C_WDAY'] = data['C_WDAY'].astype(CategoricalDtype(ordered=True))
    data['C_HOUR'] = data['C_HOUR'].astype(CategoricalDtype(ordered=True))
    data['C_VEHS'] = data['C_VEHS'].astype(CategoricalDtype(ordered=True))
    data['P_AGE'] = data['P_AGE'].astype(CategoricalDtype(ordered=True))
    data['P_PSN'] = data['P_PSN'].astype(CategoricalDtype(ordered=True))
    data['P_ISEV'] = data['P_ISEV'].astype('int')
    return data

In [3]:
#print("Sample size: {}".format(sampleN))

if multiclass:
    print("Multi-Class Classification: Enabled")
else:
    print("Multi-Class Classification: Disabled")

if enable_lr:
    print("Logistic Regression: Enabled")
else:
    print("Logistic Regression: Disabled")
    
if enable_dt:
    print("Decision Tree: Enabled")
else:
    print("Decision Tree: Disabled")
    
if enable_svm:
    print("Support Vector Machines: Enabled")
else:
    print("Support Vector Machines: Disabled")

if  enable_knn:
    print("KNN: Enabled")
else:
    print("KNN: Disabled")
    
if enable_nbayes:
    print("Naive Bayes: Enabled")
else:
    print("Naive Bayes: Disabled")
    
if enable_mlp:
    print("MLP: Enabled")
else:
    print("MLP: Disabled")

Multi-Class Classification: Disabled
Logistic Regression: Enabled
Decision Tree: Enabled
Support Vector Machines: Enabled
KNN: Enabled
Naive Bayes: Enabled
MLP: Enabled


In [4]:
#load data
df_test = pd.read_csv(datafile_test, engine = 'python')
df_train = pd.read_csv(datafile_train, engine = 'python')
df = df_train.copy()

print(df_test.head(2))
print(df_train.head(2))

   C_MNTH  C_WDAY  C_HOUR  C_VEHS  C_CONF  C_RCFG  C_WTHR  C_RSUR  C_RALN  \
0       8       5       2       2      36       2       1       1       1   
1       6       5       2       2      33       2       1       1       1   

   C_TRAF  P_SEX  P_AGE  P_PSN  P_USER  P_ISEV  
0       2      2      1      1       3       1  
1       1      1      4      1       2       0  
   C_MNTH  C_WDAY  C_HOUR  C_VEHS  C_CONF  C_RCFG  C_WTHR  C_RSUR  C_RALN  \
0       6       2       4       2      23       1       1       1       1   
1       2       6       1       1       3       1       4       6       1   

   C_TRAF  P_SEX  P_AGE  P_PSN  P_USER  P_ISEV  
0       7      2      4      1       1       1  
1       7      1      4      1       1       1  


In [5]:
df_test_cat = convert_type(df_test)
print(df_test_cat.info())
df_train_cat = convert_type(df_train)
print(df_train_cat.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1444846 entries, 0 to 1444845
Data columns (total 15 columns):
C_MNTH    1444846 non-null category
C_WDAY    1444846 non-null category
C_HOUR    1444846 non-null category
C_VEHS    1444846 non-null category
C_CONF    1444846 non-null category
C_RCFG    1444846 non-null category
C_WTHR    1444846 non-null category
C_RSUR    1444846 non-null category
C_RALN    1444846 non-null category
C_TRAF    1444846 non-null category
P_SEX     1444846 non-null category
P_AGE     1444846 non-null category
P_PSN     1444846 non-null category
P_USER    1444846 non-null category
P_ISEV    1444846 non-null int32
dtypes: category(14), int32(1)
memory usage: 24.8 MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3878536 entries, 0 to 3878535
Data columns (total 15 columns):
C_MNTH    category
C_WDAY    category
C_HOUR    category
C_VEHS    category
C_CONF    category
C_RCFG    category
C_WTHR    category
C_RSUR    category
C_RALN    category
C_TRAF  

In [6]:
total_test_Rows = df_test_cat.index.size
print("Number of Rows in test data: {}".format(total_test_Rows))

total_train_Rows = df_train_cat.index.size
print("Number of Rows in train data: {}".format(total_train_Rows))

Number of Rows in test data: 1444846
Number of Rows in train data: 3878536


In [7]:
#Split between data and class for training
Y = df_train_cat[df_train_cat.columns[-1]]
X = df_train_cat[df_train_cat.columns[0:df_train_cat.columns.size -1]]

Y_test = df_test_cat[df_test_cat.columns[-1]]
X_test = df_test_cat[df_test_cat.columns[0:df_test_cat.columns.size -1]]

In [8]:
print(Y.unique())
print(Y.groupby(Y).size())
print(Y_test.unique())
print(Y_test.groupby(Y_test).size())

[1 0]
P_ISEV
0    1939268
1    1939268
Name: P_ISEV, dtype: int64
[1 0]
P_ISEV
0    613731
1    831115
Name: P_ISEV, dtype: int64


   ### Logistic Regression Model Evaluation with cross fold

In [12]:
if enable_lr:
    print("Logistic Regression: Start")
    t_start =  time.time()
    print()
    print("Model: Logistic Regression")
    print(time.asctime( time.localtime(t_start) ))
    print()
    print("Using Kfold: {}".format(nfold))
    #kfold = model_selection.KFold(n_splits=nfold)
    kfold = model_selection.StratifiedKFold(n_splits=nfold, shuffle=True)  
    
    lr = LogisticRegression(C=1, random_state=0, solver='saga', multi_class='ovr', 
                            verbose=verbose_level, n_jobs=10, max_iter=model_max_iter)
    print()
    print("Model: Logistic Regression")
    print(lr)
    print()
    print("Logistic Regression: Fit")
    results = model_selection.cross_val_score(lr, X, Y, cv=kfold)
    
    print('Kfold results: {}'.format(results))
    print('Kfold mean" {}'.format(results.mean()))

    
    t_end =  time.time()
    print(time.asctime( time.localtime(t_end) ))
    
print("Logistic Regression: End")

Logistic Regression: Start

Model: Logistic Regression
Fri Dec  7 17:39:50 2018

Using Kfold: 10

Model: Logistic Regression
LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=1000, multi_class='ovr', n_jobs=10,
          penalty='l2', random_state=0, solver='saga', tol=0.0001,
          verbose=3, warm_start=False)

Logistic Regression: Fit
convergence after 16 epochs took 40 seconds


[Parallel(n_jobs=10)]: Done   1 out of   1 | elapsed:   40.3s finished


convergence after 14 epochs took 35 seconds


[Parallel(n_jobs=10)]: Done   1 out of   1 | elapsed:   35.4s finished


convergence after 15 epochs took 35 seconds


...

convergence after 15 epochs took 34 seconds


[Parallel(n_jobs=10)]: Done   1 out of   1 | elapsed:   33.9s finished


convergence after 15 epochs took 37 seconds


[Parallel(n_jobs=10)]: Done   1 out of   1 | elapsed:   37.6s finished


Kfold results: [0.62984267 0.62883714 0.62879073 0.62945593 0.62717414 0.62913622
 0.62849681 0.62947398 0.62968864 0.62696853]
Kfold mean" 0.6287864797873317
Fri Dec  7 17:46:18 2018
Logistic Regression: End


### Naive Bayes Model Evaluation with cross fold

In [13]:
if enable_nbayes:
    print("Naive Bayes: Start")
    t_start =  time.time()
    print()
    print("Model: Naive Bayes")
    print(time.asctime( time.localtime(t_start) ))
    print()
    print("Using Stratified Kfold: {}".format(nfold))
    kfold = model_selection.StratifiedKFold(n_splits=nfold, shuffle=True)  
    nbayes = GaussianNB()
    print()
    print("Model: Naive Bayes")
    print(nbayes)
    print()
    print("Naive Bayes: Fit")
    results = model_selection.cross_val_score(nbayes, X, Y, cv=kfold)
    #lr.fit(X_train, Y_train)
    
    print('Kfold results: {}'.format(results))
    print('Kfold mean" {}'.format(results.mean()))

    
    t_end =  time.time()
    print(time.asctime( time.localtime(t_end) ))
    
print("Naive Bayes: End")

Naive Bayes: Start

Model: Naive Bayes
Fri Dec  7 17:46:18 2018

Using Stratified Kfold: 10

Model: Naive Bayes
LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=1000, multi_class='ovr', n_jobs=10,
          penalty='l2', random_state=0, solver='saga', tol=0.0001,
          verbose=3, warm_start=False)

Naive Bayes: Fit
Kfold results: [0.5989032  0.59893671 0.60032641 0.59868662 0.59991904 0.59963285
 0.60000413 0.59924353 0.59917443 0.59801935]
Kfold mean" 0.5992846262736881
Fri Dec  7 17:46:59 2018
Naive Bayes: End


### Decision Tree Model Evaluation with cross fold

In [14]:
if predict_dt:
    print("Decision Tree: Start")
    t_start =  time.time()
    print()
    print("Model: Decision Tree")
    print(time.asctime( time.localtime(t_start) ))
    print()
    print("Using Stratified Kfold: {}".format(nfold))
    kfold = model_selection.StratifiedKFold(n_splits=nfold, shuffle=True)  
    tree = DecisionTreeClassifier(criterion='entropy',max_depth=50)
    print()
    print("Model: Decision Tree")
    print(tree)
    print()
    print("Decision Tree: Fit")
    results = model_selection.cross_val_score(tree, X, Y, cv=kfold)
    #lr.fit(X_train, Y_train)
    
    print('Kfold results: {}'.format(results))
    print('Kfold mean" {}'.format(results.mean()))

    
    t_end =  time.time()
    print(time.asctime( time.localtime(t_end) ))
    
print("Decision Tree: End")

Decision Tree: Start

Model: Decision Tree
Fri Dec  7 17:46:59 2018

Using Stratified Kfold: 10

Model: Decision Tree
LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=1000, multi_class='ovr', n_jobs=10,
          penalty='l2', random_state=0, solver='saga', tol=0.0001,
          verbose=3, warm_start=False)

Decision Tree: Fit
Kfold results: [0.65743811 0.65820902 0.65795892 0.65858803 0.65695597 0.65817808
 0.65740717 0.65688378 0.65736931 0.65749306]
Kfold mean" 0.6576481433025675
Fri Dec  7 17:54:43 2018
Decision Tree: End


### ANN - Multilayer Perceptron Model evaluation with cross fold

In [15]:
if predict_mlp:
    print("Multilayer Perceptron: Start")
    t_start =  time.time()
    print()
    print("Model: Multilayer Perceptron")
    print(time.asctime( time.localtime(t_start) ))
    print()
    print("Using Stratified Kfold: {}".format(nfold))
    kfold = model_selection.StratifiedKFold(n_splits=nfold, shuffle=True)  
    mlpc = MLPClassifier(hidden_layer_sizes=(25, 25, 25), verbose=verbose_level, max_iter=model_max_iter)
    print()
    print("Model: Multilayer Perceptron")
    print(mlpc)
    print()
    print("Multilayer Perceptron: Fit")
    results = model_selection.cross_val_score(mlpc, X, Y, cv=kfold)
    #lr.fit(X_train, Y_train)
    
    print('Kfold results: {}'.format(results))
    print('Kfold mean" {}'.format(results.mean()))

    
    t_end =  time.time()
    print(time.asctime( time.localtime(t_end) ))
    
print("Multilayer Perceptron: End")

Multilayer Perceptron: Start

Model: Multilayer Perceptron
Fri Dec  7 17:54:43 2018

Using Stratified Kfold: 10

Model: Multilayer Perceptron
LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=1000, multi_class='ovr', n_jobs=10,
          penalty='l2', random_state=0, solver='saga', tol=0.0001,
          verbose=3, warm_start=False)

Multilayer Perceptron: Fit
Iteration 1, loss = 0.60978339
Iteration 2, loss = 0.60219070
Iteration 3, loss = 0.60031652
Iteration 4, loss = 0.59918413
Iteration 5, loss = 0.59791984
Iteration 6, loss = 0.59741047
Iteration 7, loss = 0.59705158
Iteration 8, loss = 0.59685187
Iteration 9, loss = 0.59662792
Iteration 10, loss = 0.59644359
Iteration 11, loss = 0.59630366
Iteration 12, loss = 0.59620469
Iteration 13, loss = 0.59602715
Iteration 14, loss = 0.59602250
Iteration 15, loss = 0.59586309
Iteration 16, loss = 0.59574551
Iteration 17, loss = 0.59565466
Iteration 18, loss = 0.59558321
Iterat

### SVM Model Evaluation with SVM

In [16]:
if predict_svm:
    print("SVM: Start")
    t_start =  time.time()
    print()
    print("Model: SVM")
    print(time.asctime( time.localtime(t_start) ))
    print()
    print("Using Stratified Kfold: {}".format(nfold))
    kfold = model_selection.StratifiedKFold(n_splits=nfold, shuffle=True)  
    svm = SVC(C=1, gamma = 'auto', verbose = verbose_level, max_iter=model_max_iter)
    print()
    print("Model: SVM")
    print(svm)
    print()
    print("SVM: Fit")
    results = model_selection.cross_val_score(svm, X, Y, cv=kfold)
    #lr.fit(X_train, Y_train)
    
    print('Kfold results: {}'.format(results))
    print('Kfold mean" {}'.format(results.mean()))

    
    t_end =  time.time()
    print(time.asctime( time.localtime(t_end) ))
    
print("SVM: End")

SVM: Start

Model: SVM
Fri Dec  7 19:34:12 2018

Using Stratified Kfold: 10

Model: SVM
LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=1000, multi_class='ovr', n_jobs=10,
          penalty='l2', random_state=0, solver='saga', tol=0.0001,
          verbose=3, warm_start=False)

SVM: Fit
[LibSVM]

C:\Users\pradeep\Anaconda3\lib\site-packages\sklearn\svm\base.py:218: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


[LibSVM]

C:\Users\pradeep\Anaconda3\lib\site-packages\sklearn\svm\base.py:218: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


[LibSVM]

...

[LibSVM]

C:\Users\pradeep\Anaconda3\lib\site-packages\sklearn\svm\base.py:218: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


[LibSVM]

C:\Users\pradeep\Anaconda3\lib\site-packages\sklearn\svm\base.py:218: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


Kfold results: [0.49676682 0.49743202 0.49750679 0.49841435 0.49470677 0.49593146
 0.49304377 0.49518375 0.4948073  0.49814362]
Kfold mean" 0.496193667179361
Fri Dec  7 20:13:04 2018
SVM: End


### K-N-N Model Evaluation with cross fold

In [17]:
if predict_knn:
    print("KNN: Start")
    t_start =  time.time()
    print()
    print("Model: KNN")
    print(time.asctime( time.localtime(t_start) ))
    print()
    print("Using Stratified Kfold: {}".format(nfold))
    kfold = model_selection.StratifiedKFold(n_splits=nfold, shuffle=True)  
    knn = KNeighborsClassifier(n_neighbors=5, p=2, metric='minkowski', n_jobs = -1)
    print()
    print("Model: KNN")
    print(knn)
    print()
    print("KNN: Fit")
    results = model_selection.cross_val_score(knn, X, Y, cv=kfold)
    #lr.fit(X_train, Y_train)
    
    print('Kfold results: {}'.format(results))
    print('Kfold mean" {}'.format(results.mean()))

    
    t_end =  time.time()
    print(time.asctime( time.localtime(t_end) ))
    
print("KNN: End")

KNN: End


In [ ]:
## extra code and play

In [18]:
if (False):    
    seed = 101
    print("Logistic Regression: Start")
    t_start =  time.time()
    print()
    print("Model: Logistic Regression")
    print(time.asctime( time.localtime(t_start) ))
    print()
    print("Using Kfold: {}".format(nfold))
    #kfold = model_selection.StratifiedKFold(n_splits=nfold, shuffle=True, random_state=seed)
    kfold = model_selection.KFold(n_splits=nfold, random_state=seed)
    
    models = []
    model_scores_train = []
    model_scores_val = []
    
    X_kfold = np.array(X)
    Y_kfold = np.array(Y)
    
    i = 0
    for train_index, val_index in kfold.split(X_kfold):
        i = i+1
        print("Fold: {}".format(i))
        
        X_train, X_val = X_kfold[train_index], X_kfold[val_index]
        y_train, y_val = Y_kfold[train_index], Y_kfold[val_index]
        
        lr = LogisticRegression(C=1, random_state=0, solver='saga', multi_class='ovr', 
                            verbose=verbose_level, n_jobs=10, max_iter=model_max_iter)
        print()
        print("Model: Logistic Regression")
        print(lr)
        print()
        print("Logistic Regression: Fit")
        lr.fit(X_train, y_train)

        #add model for list
        models.append(lr)

        print("Logistic Regression: Predict")
        y_pred = lr.predict(X_val)

        mst = lr.score(X_train, y_train)
        model_scores_train.append(mst)
        
        msv = lr.score(X_val, y_val)
        model_scores_val.append(msv)
        
        print('Accuracy of logistic regression classifier on train set: {:.2f}'.format(mst))
        print('Accuracy of logistic regression classifier on validation set: {:.2f}'.format(msv))

        # print the intercept (Note: one vs rest => 1 vs 2and3, 2 vs 1and3, 3 vs 1and2)
        print("Logistic Regression: Intercept")
        print(lr.intercept_)

        # print the coeficients (Note: one vs rest => 1 vs 2and3, 2 vs 1and3, 3 vs 1and2)
        print("Logistic Regression: Coefficients")
        print(lr.coef_)

        print("Logistic Regression: Confusion Matrix")
        cnf_matrix_lg = confusion_matrix(y_val, y_pred)
        print(cnf_matrix_lg)
    
        print("Logistic Regression: Classification Report")
        print(classification_report(y_val, y_pred))

    print('AVG Accuracy of logistic regression classifier on train set: {:.2f}'.format(np.mean(model_scores_train)))
    print('AVG Accuracy of logistic regression classifier on validation set: {:.2f}'.format(np.mean(model_scores_val)))
    
    t_end =  time.time()
    print(time.asctime( time.localtime(t_end) ))
    
    # save model to file
    pickle.dump(models, open(file_lr, "wb"))
    print("Logistic Regression: End")

In [19]:
if (True):    
    seed = 101
    print("Logistic Regression: Start")
    t_start =  time.time()
    print()
    print("Model: Logistic Regression")
    print(time.asctime( time.localtime(t_start) ))
    print()
    print("Using Stratified Kfold: {}".format(nfold))
    
    models = []
    model_scores_train = []
    model_scores_val = []
    
    X_kfold = np.array(X)
    Y_kfold = np.array(Y)
    
    skf = model_selection.StratifiedKFold(n_splits=nfold, shuffle=True, random_state=seed)    
    i = 0
    for train_index, val_index in skf.split(X_kfold, Y_kfold):
        i = i+1
        print("Fold: {}".format(i))
        
        X_train, X_val = X_kfold[train_index], X_kfold[val_index]
        y_train, y_val = Y_kfold[train_index], Y_kfold[val_index]
        
        lr = LogisticRegression(C=1, random_state=0, solver='saga', multi_class='ovr', 
                            verbose=verbose_level, n_jobs=10, max_iter=model_max_iter)
        print()
        print("Model: Logistic Regression")
        print(lr)
        print()
        print("Logistic Regression: Fit")
        lr.fit(X_train, y_train)

        #add model for list
        models.append(lr)

        print("Logistic Regression: Predict")
        y_pred = lr.predict(X_val)

        mst = lr.score(X_train, y_train)
        model_scores_train.append(mst)
        
        msv = lr.score(X_val, y_val)
        model_scores_val.append(msv)
        
        print('Accuracy of logistic regression classifier on train set: {:.2f}'.format(mst))
        print('Accuracy of logistic regression classifier on validation set: {:.2f}'.format(msv))

        # print the intercept (Note: one vs rest => 1 vs 2and3, 2 vs 1and3, 3 vs 1and2)
        print("Logistic Regression: Intercept")
        print(lr.intercept_)

        # print the coeficients (Note: one vs rest => 1 vs 2and3, 2 vs 1and3, 3 vs 1and2)
        print("Logistic Regression: Coefficients")
        print(lr.coef_)

        print("Logistic Regression: Confusion Matrix")
        cnf_matrix_lg = confusion_matrix(y_val, y_pred)
        print(cnf_matrix_lg)
    
        print("Logistic Regression: Classification Report")
        print(classification_report(y_val, y_pred))

    print('AVG Accuracy of logistic regression classifier on train set: {:.2f}'.format(np.mean(model_scores_train)))
    print('AVG Accuracy of logistic regression classifier on validation set: {:.2f}'.format(np.mean(model_scores_val)))
    
    t_end =  time.time()
    print(time.asctime( time.localtime(t_end) ))
    
    # save model to file
    pickle.dump(models, open(file_lr, "wb"))
    print("Logistic Regression: End")

Logistic Regression: Start

Model: Logistic Regression
Fri Dec  7 20:13:04 2018

Using Stratified Kfold: 10
Fold: 1

Model: Logistic Regression
LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=1000, multi_class='ovr', n_jobs=10,
          penalty='l2', random_state=0, solver='saga', tol=0.0001,
          verbose=3, warm_start=False)

Logistic Regression: Fit
convergence after 15 epochs took 31 seconds
Logistic Regression: Predict


[Parallel(n_jobs=10)]: Done   1 out of   1 | elapsed:   31.8s finished


Accuracy of logistic regression classifier on train set: 0.63
Accuracy of logistic regression classifier on validation set: 0.63
Logistic Regression: Intercept
[0.04576488]
Logistic Regression: Coefficients
[[ 0.00095438 -0.00998305 -0.06217297 -0.42946142 -0.00333273 -0.04236027
   0.0315637   0.0450024   0.19386947  0.04082231 -0.55242555  0.17658465
  -0.14325497  0.7504041 ]]
Logistic Regression: Confusion Matrix
[[129192  64735]
 [ 79293 114634]]
Logistic Regression: Classification Report
             precision    recall  f1-score   support

          0       0.62      0.67      0.64    193927
          1       0.64      0.59      0.61    193927

avg / total       0.63      0.63      0.63    387854

Fold: 2

Model: Logistic Regression
LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=1000, multi_class='ovr', n_jobs=10,
          penalty='l2', random_state=0, solver='saga', tol=0.0001,
          verbose=3, warm_start=

[Parallel(n_jobs=10)]: Done   1 out of   1 | elapsed:   33.9s finished


Accuracy of logistic regression classifier on train set: 0.63
Accuracy of logistic regression classifier on validation set: 0.63
Logistic Regression: Intercept
[0.05013813]
Logistic Regression: Coefficients
[[ 0.00089065 -0.01029917 -0.06168978 -0.43099227 -0.00336988 -0.0415294
   0.03150169  0.04472478  0.19434951  0.04111407 -0.55317587  0.17637422
  -0.14284058  0.74935495]]
Logistic Regression: Confusion Matrix
[[128961  64966]
 [ 79312 114615]]
Logistic Regression: Classification Report
             precision    recall  f1-score   support

          0       0.62      0.66      0.64    193927
          1       0.64      0.59      0.61    193927

avg / total       0.63      0.63      0.63    387854

Fold: 3

Model: Logistic Regression
LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=1000, multi_class='ovr', n_jobs=10,
          penalty='l2', random_state=0, solver='saga', tol=0.0001,
          verbose=3, warm_start=F

...

Accuracy of logistic regression classifier on train set: 0.63
Accuracy of logistic regression classifier on validation set: 0.63
Logistic Regression: Intercept
[0.04382782]
Logistic Regression: Coefficients
[[ 0.00113226 -0.01038706 -0.06085607 -0.42984322 -0.00330198 -0.04164114
   0.03099876  0.04532493  0.19323669  0.0411025  -0.55387355  0.17648542
  -0.14229189  0.7490564 ]]
Logistic Regression: Confusion Matrix
[[129435  64492]
 [ 79241 114686]]
Logistic Regression: Classification Report
             precision    recall  f1-score   support

          0       0.62      0.67      0.64    193927
          1       0.64      0.59      0.61    193927

avg / total       0.63      0.63      0.63    387854

Fold: 9

Model: Logistic Regression
LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=1000, multi_class='ovr', n_jobs=10,
          penalty='l2', random_state=0, solver='saga', tol=0.0001,
          verbose=3, warm_start=

[Parallel(n_jobs=10)]: Done   1 out of   1 | elapsed:   29.3s finished


Accuracy of logistic regression classifier on train set: 0.63
Accuracy of logistic regression classifier on validation set: 0.63
Logistic Regression: Intercept
[0.04627982]
Logistic Regression: Coefficients
[[ 0.00096166 -0.01036014 -0.06157487 -0.42962562 -0.00335257 -0.04228044
   0.03163171  0.04472327  0.1932632   0.04093215 -0.55163691  0.17630661
  -0.14343212  0.75020885]]
Logistic Regression: Confusion Matrix
[[129560  64366]
 [ 79391 114535]]
Logistic Regression: Classification Report
             precision    recall  f1-score   support

          0       0.62      0.67      0.64    193926
          1       0.64      0.59      0.61    193926

avg / total       0.63      0.63      0.63    387852

Fold: 10

Model: Logistic Regression
LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=1000, multi_class='ovr', n_jobs=10,
          penalty='l2', random_state=0, solver='saga', tol=0.0001,
          verbose=3, warm_start

[Parallel(n_jobs=10)]: Done   1 out of   1 | elapsed:   31.6s finished


Accuracy of logistic regression classifier on train set: 0.63
Accuracy of logistic regression classifier on validation set: 0.63
Logistic Regression: Intercept
[0.04714239]
Logistic Regression: Coefficients
[[ 6.93809325e-04 -1.04109404e-02 -6.11208849e-02 -4.30097449e-01
  -3.38960347e-03 -4.02136947e-02  3.15704008e-02  4.51577526e-02
   1.93960781e-01  4.08899195e-02 -5.52811195e-01  1.76115812e-01
  -1.43057777e-01  7.49398909e-01]]
Logistic Regression: Confusion Matrix
[[129312  64614]
 [ 79387 114539]]
Logistic Regression: Classification Report
             precision    recall  f1-score   support

          0       0.62      0.67      0.64    193926
          1       0.64      0.59      0.61    193926

avg / total       0.63      0.63      0.63    387852

AVG Accuracy of logistic regression classifier on train set: 0.63
AVG Accuracy of logistic regression classifier on validation set: 0.63
Fri Dec  7 20:18:58 2018
Logistic Regression: End


In [20]:
## for each model generated, lets predict the on test set
## note, this is the first time any of these observations are seen by the model
# load model from file
loaded_model = pickle.load(open(file_lr, "rb"))
print("Logistic Regression: Predict on test set")

i = 0
for model in loaded_model:
    i = i + 1
    y_pred = model.predict(X_test)

    print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(model.score(X_test, Y_test)))

    # print the intercept (Note: one vs rest => 1 vs 2and3, 2 vs 1and3, 3 vs 1and2)
    print("Logistic Regression: Intercept")
    print(model.intercept_)

    # print the coeficients (Note: one vs rest => 1 vs 2and3, 2 vs 1and3, 3 vs 1and2)
    print("Logistic Regression: Coefficients")
    print(model.coef_)

    print("Logistic Regression: Confusion Matrix")
    cnf_matrix_lg = confusion_matrix(Y_test, y_pred)
    print(cnf_matrix_lg)
    
    print("Logistic Regression: Classification Report")
    print(classification_report(Y_test, y_pred))

Logistic Regression: Predict on test set
Accuracy of logistic regression classifier on test set: 0.62
Logistic Regression: Intercept
[0.04576488]
Logistic Regression: Coefficients
[[ 0.00095438 -0.00998305 -0.06217297 -0.42946142 -0.00333273 -0.04236027
   0.0315637   0.0450024   0.19386947  0.04082231 -0.55242555  0.17658465
  -0.14325497  0.7504041 ]]
Logistic Regression: Confusion Matrix
[[406751 206980]
 [339165 491950]]
Logistic Regression: Classification Report
             precision    recall  f1-score   support

          0       0.55      0.66      0.60    613731
          1       0.70      0.59      0.64    831115

avg / total       0.64      0.62      0.62   1444846

Accuracy of logistic regression classifier on test set: 0.62
Logistic Regression: Intercept
[0.05013813]
Logistic Regression: Coefficients
[[ 0.00089065 -0.01029917 -0.06168978 -0.43099227 -0.00336988 -0.0415294
   0.03150169  0.04472478  0.19434951  0.04111407 -0.55317587  0.17637422
  -0.14284058  0.74935495]]

In [21]:
if enable_lr:
    seed = 101
    print("Logistic Regression: Start")
    t_start =  time.time()
    print()
    print("Model: Logistic Regression")
    print(time.asctime( time.localtime(t_start) ))
    print()
    print("Using Kfold: {}".format(nfold))
    kfold = model_selection.KFold(n_splits=nfold, random_state=seed)
    
    lr = LogisticRegression(C=1, random_state=0, solver='saga', multi_class='ovr', 
                            verbose=verbose_level, n_jobs=10, max_iter=model_max_iter)
    print()
    print("Model: Logistic Regression")
    print(lr)
    print()
    print("Logistic Regression: Fit")
    results = model_selection.cross_val_score(lr, X, Y, cv=kfold)
    #lr.fit(X_train, Y_train)
    
    print(results.mean())
    print(results)
    
    t_end =  time.time()
    print(time.asctime( time.localtime(t_end) ))
    
print("Logistic Regression: End")

Logistic Regression: Start

Model: Logistic Regression
Fri Dec  7 20:19:27 2018

Using Kfold: 10

Model: Logistic Regression
LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=1000, multi_class='ovr', n_jobs=10,
          penalty='l2', random_state=0, solver='saga', tol=0.0001,
          verbose=3, warm_start=False)

Logistic Regression: Fit
convergence after 15 epochs took 31 seconds


[Parallel(n_jobs=10)]: Done   1 out of   1 | elapsed:   31.4s finished


convergence after 16 epochs took 34 seconds


[Parallel(n_jobs=10)]: Done   1 out of   1 | elapsed:   33.9s finished


convergence after 15 epochs took 32 seconds


...

convergence after 15 epochs took 31 seconds


[Parallel(n_jobs=10)]: Done   1 out of   1 | elapsed:   31.7s finished


convergence after 15 epochs took 32 seconds


[Parallel(n_jobs=10)]: Done   1 out of   1 | elapsed:   31.7s finished


0.6099698316722773
[0.6178923  0.61885142 0.61766025 0.61714975 0.61933614 0.61787425
 0.61912116 0.61704563 0.63146862 0.52329878]
Fri Dec  7 20:25:17 2018
Logistic Regression: End
